In [ ]:
!pip install -q transformers torch sentencepiece

In [ ]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

model_name = "tuner007/pegasus_paraphrase"

tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

In [ ]:
def get_response(input_text, num_return_sequences, num_beans):
    batch = tokenizer([input_text], truncation=True, padding="longest", return_tensors="pt").to(device)
    translated = model.generate(**batch, num_beans=num_beans, num_return_sequences=num_return_sequences, temperature=0.4)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)

    return tgt_text